In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("catalog","catalog_ecommerce")
dbutils.widgets.text("schema_source","silver")
dbutils.widgets.text("shema_sink","gold")

In [0]:
catalog = dbutils.widgets.get("catalog")
schema_source = dbutils.widgets.get("schema_source")
schema_sink = dbutils.widgets.get("shema_sink")

#Metricas

In [0]:
df_sales_category = spark.sql(f"""
        SELECT 
            p.category_english, 
            -- Metricas de ventas
            count(DISTINCT oi.order_id) as Total_orders,
            count(oi.order_item_id) as Items_sold,

            -- Metricas revenue
            ROUND(SUM(oi.Total_pay),2) as Total_revenue


        FROM  {catalog}.{schema_source}.productos AS p
        INNER JOIN {catalog}.{schema_source}.orders_items AS oi
            ON p.product_id = oi.product_id
        GROUP BY p.category_english
        ORDER BY Total_revenue DESC
""")

In [0]:
df_sales_category.write.mode("overwrite").saveAsTable(f"{catalog}.{schema_sink}.sales_by_category")